# BicycleGAN

## 1. Import Libs

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

from dataloader import Edges2handbags
import util
import os
import argparse

from model import *

CUDA = torch.cuda.is_available()

## 2. Settings

In [2]:
parser = argparse.ArgumentParser([])

parser.add_argument('--root', required=False, default='./dataset/edges2handbags/edges2handbags')
parser.add_argument('--result_dir', default='result/edges2handbags')
parser.add_argument('--weight_dir', default='weight/edges2handbags')
parser.add_argument('--load_weight', type=bool, default=False)
parser.add_argument('--batch_size', type=int, default=2)
parser.add_argument('--test_size', type=int, default=20)
parser.add_argument('--test_img_num', type=int, default=5)
parser.add_argument('--img_size', type=int, default=128)
parser.add_argument('--num_epoch', type=int, default=50)
parser.add_argument('--save_every', type=int, default=1000)
parser.add_argument('--lr', type=float, default=0.0002)
parser.add_argument('--beta_1', type=float, default=0.5)
parser.add_argument('--beta_2', type=float, default=0.999)
parser.add_argument('--lambda_kl', type=float, default=0.01)
parser.add_argument('--lambda_img', type=int, default=10)
parser.add_argument('--lambda_z', type=float, default=0.5)
parser.add_argument('--z_dim', type=int, default=8)

params = parser.parse_args([])
print(params)

Namespace(batch_size=2, beta_1=0.5, beta_2=0.999, img_size=128, lambda_img=10, lambda_kl=0.01, lambda_z=0.5, load_weight=False, lr=0.0002, num_epoch=50, result_dir='result/edges2handbags', root='./dataset/edges2handbags/edges2handbags', save_every=1000, test_img_num=5, test_size=20, weight_dir='weight/edges2handbags', z_dim=8)


## 3. Utils

In [3]:
def mse_loss(score, target=1):
    dtype = type(score)
    
    if target == 1:
        label = util.var(torch.ones(score.size()), requires_grad=False)
    elif target == 0:
        label = util.var(torch.zeros(score.size()), requires_grad=False)
    
    criterion = nn.MSELoss().cuda()
    loss = criterion(score, label)
    
    return loss

def L1_loss(pred, target):
    return torch.mean(torch.abs(pred - target))

def lr_decay_rule(epoch, start_decay=100, lr_decay=100):
    decay_rate = 1.0 - (max(0, epoch - start_decay) / float(lr_decay))
    return decay_rate

## 4. Load Dataset
### 4.1 Preprocessing

In [4]:
transform = transforms.Compose([
    transforms.Scale((params.img_size, params.img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

### 4.2 DataLoader

In [5]:
train_dataset = Edges2handbags(params.root, transform=transform, mode='train')
test_dataset = Edges2handbags(params.root, transform=transform, mode='val')

train_data_loader = DataLoader(train_dataset, batch_size=params.batch_size, shuffle=True, num_workers=4, drop_last=True)
test_data_loader = DataLoader(test_dataset, batch_size=params.batch_size, shuffle=True, num_workers=4, drop_last=True)

train_sample = train_dataset.__getitem__(11)
print(train_sample)
print(len(train_sample))

(
( 0 ,.,.) = 
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
           ...             ⋱             ...          
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000

( 1 ,.,.) = 
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
           ...             ⋱             ...          
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000

( 2 ,.,.) = 
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
           ...        

## 5. Build Models & Optimizers & Loss Function

### 5.1 Models

In [6]:
D_cVAE = Discriminator().cuda()
D_cLR = Discriminator().cuda()
G = Generator(z_dim=params.z_dim).cuda()
E = Encoder(z_dim=params.z_dim).cuda()

print(D_cVAE)
print(D_cLR)
print(G)
print(E)

Discriminator(
  (d_1): Sequential(
    (0): AvgPool2d(kernel_size=3, stride=2, padding=0, ceil_mode=False, count_include_pad=False)
    (1): ConvBlock(
      (conv_block): Sequential(
        (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): LeakyReLU(0.2, inplace)
      )
    )
    (2): ConvBlock(
      (conv_block): Sequential(
        (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (3): ConvBlock(
      (conv_block): Sequential(
        (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
        (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (4): ConvBlock(
      (conv_block): Sequential(
        (0): Conv2d(128, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
      )
    )
  )
  (d_2): Sequential(
    (0): ConvBlock(
      (conv_block): Sequential(
        (0): Conv2d(3, 64, kerne

### 5.2 Optimizers

In [7]:
optim_D_cVAE = optim.Adam(D_cVAE.parameters(), lr=params.lr, betas=(params.beta_1, params.beta_2))
optim_D_cLR = optim.Adam(D_cLR.parameters(), lr=params.lr, betas=(params.beta_1, params.beta_2))
optim_G = optim.Adam(G.parameters(), lr=params.lr, betas=(params.beta_1, params.beta_2))
optim_E = optim.Adam(E.parameters(), lr=params.lr, betas=(params.beta_1, params.beta_2))

In [8]:
# 20 x 5 x 8
fixed_z = util.var(torch.randn(params.test_size, params.test_img_num, params.z_dim)).cuda()

## 6. Load Pretrained Models

In [9]:
if params.load_weight is True:
    D_cVAE.load_state_dict(torch.load(os.path.join(params.weight_dir, 'D_cVAE.pkl')))
    D_cLR.load_state_dict(torch.load(os.path.join(params.weight_dir, 'D_cLR.pkl')))
    G.load_state_dict(torch.load(os.path.join(params.weight_dir, 'G.pkl')))
    E.load_state_dict(torch.load(os.path.join(params.weight_dir, 'E.pkl')))

In [ ]:
D_cVAE.train()
D_cLR.train()
G.train()
E.train()

Encoder(
  (conv): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (res_blocks): Sequential(
    (0): ResBlock(
      (conv): Sequential(
        (0): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (1): LeakyReLU(0.2, inplace)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (4): LeakyReLU(0.2, inplace)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): AvgPool2d(kernel_size=2, stride=2, padding=0, ceil_mode=False, count_include_pad=True)
      )
      (short_cut): Sequential(
        (0): AvgPool2d(kernel_size=2, stride=2, padding=0, ceil_mode=False, count_include_pad=True)
        (1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (1): ResBlock(
      (conv): Sequential(
        (0): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        (1): LeakyReLU(0.2, inplace)


In [ ]:
for epoch in range(1, params.num_epoch + 1):
    for iters, (img, ground_truth) in enumerate(train_data_loader): # 1
        # img : (2, 3, 128, 128) of domain A / ground_truth : (2, 3, 128, 128) of domain B
        img, ground_truth = util.var(img), util.var(ground_truth)
        
        # Seperate data for cVAE_GAN and cLR_GAN
        cVAE_data = {'img' : img[0].unsqueeze(dim=0), 'ground_truth' : ground_truth[0].unsqueeze(dim=0)}
        cLR_data = {'img' : img[1].unsqueeze(dim=0), 'ground_truth' : ground_truth[1].unsqueeze(dim=0)}
        
        ''' ----------------------------- 1. Train D ----------------------------- '''
        #############   Step 1. D loss in cVAE-GAN #############
        
        # Encoded latent vector
        mu, log_variance = E(cVAE_data['ground_truth'])
        std = torch.exp(log_variance / 2)
        random_z = util.var(torch.randn(1, params.z_dim))
        encoded_z = (random_z * std) + mu
        
        # Generate fake image
        fake_img_cVAE = G(cVAE_data['img'], encoded_z)
        
        # Get scores and loss
        real_d_cVAE_1, real_d_cVAE_2 = D_cVAE(cVAE_data['ground_truth'])
        fake_d_cVAE_1, fake_d_cVAE_2 = D_cVAE(fake_img_cVAE)
                
        # mse_loss for LSGAN
        D_loss_cVAE_1 = mse_loss(real_d_cVAE_1, 1) + mse_loss(fake_d_cVAE_1, 0)
        D_loss_cVAE_2 = mse_loss(real_d_cVAE_2, 1) + mse_loss(fake_d_cVAE_2, 0)
                
        #############   Step 2. D loss in cLR-GAN   #############

        # Random latent vector
        random_z = util.var(torch.randn(1, params.z_dim))

        # Generate fake image
        fake_img_cLR = G(cLR_data['img'], random_z)

        # Get scores and loss
        real_d_cLR_1, real_d_cLR_2 = D_cLR(cLR_data['ground_truth'])
        fake_d_cLR_1, fake_d_cLR_2 = D_cLR(fake_img_cLR)
        
        D_loss_cLR_1 = mse_loss(real_d_cLR_1, 1) + mse_loss(fake_d_cLR_1, 0)
        D_loss_cLR_2 = mse_loss(real_d_cLR_2, 1) + mse_loss(fake_d_cLR_2, 0)
        
        D_loss = D_loss_cVAE_1 + D_loss_cLR_1 + D_loss_cVAE_2 + D_loss_cLR_2
        
        # Update
        optim_D_cVAE.zero_grad()
        optim_D_cLR.zero_grad()
        optim_G.zero_grad()
        optim_E.zero_grad()
        D_loss.backward()
        optim_D_cVAE.step()
        optim_D_cLR.step()
        
        ''' ----------------------------- 2. Train G & E ----------------------------- '''
        ############# Step 1. GAN loss to fool discriminator (cVAE_GAN and cLR_GAN) #############
        
        # Encoded latent vector
        mu, log_variance = E(cVAE_data['ground_truth'])
        std = torch.exp(log_variance / 2)
        random_z = util.var(torch.randn(1, params.z_dim))
        encoded_z = (random_z * std) + mu
        
        # Generate fake image and get adversarial loss
        fake_img_cVAE = G(cVAE_data['img'], encoded_z)
        fake_d_cVAE_1, fake_d_cVAE_2 = D_cVAE(fake_img_cVAE)
        
        GAN_loss_cVAE_1 = mse_loss(fake_d_cVAE_1, 1)
        GAN_loss_cVAE_2 = mse_loss(fake_d_cVAE_2, 1)
        
        # Random latent vector
        random_z = util.var(torch.randn(1, params.z_dim))
        
        # Generate fake image and get adversarial loss
        fake_img_cLR = G(cLR_data['img'], random_z)
        fake_d_cLR_1, fake_d_cLR_2 = D_cLR(fake_img_cLR)
        
        GAN_loss_cLR_1 = mse_loss(fake_d_cLR_1, 1)
        GAN_loss_cLR_2 = mse_loss(fake_d_cLR_2, 1)
        
        G_GAN_loss = GAN_loss_cVAE_1 + GAN_loss_cVAE_2 + GAN_loss_cLR_1 + GAN_loss_cLR_2
        
        ############# Step 2. KL-divergence with N(0, 1) (cVAE-GAN) #############
                
        KL_div = params.lambda_kl * torch.sum(0.5 * (mu ** 2 + torch.exp(log_variance) - log_variance - 1))
        
        ############# Step 3. Reconstruction of ground truth image (|G(A, z) - B|) (cVAE-GAN) #############
        img_recon_loss = params.lambda_img * L1_loss(fake_img_cVAE, cVAE_data['ground_truth'])
        
        EG_loss = G_GAN_loss + KL_div + img_recon_loss
        optim_D_cVAE.zero_grad()
        optim_D_cLR.zero_grad()
        optim_G.zero_grad()
        optim_E.zero_grad()
        EG_loss.backward(retain_graph=True)
        optim_E.step()
        optim_G.step()
        
        ''' ----------------------------- 3. Train ONLY G ----------------------------- '''
        ############ Step 1. Reconstrution of random latent code (|E(G(A, z)) - z|) (cLR-GAN) ############
                
        # This step should update ONLY G.
        mu_, log_variance_ = E(fake_img_cLR)
        z_recon_loss = L1_loss(mu_, random_z)
        
        G_alone_loss = params.lambda_z * z_recon_loss
        
        optim_D_cVAE.zero_grad()
        optim_D_cLR.zero_grad()
        optim_G.zero_grad()
        optim_E.zero_grad()
        G_alone_loss.backward()
        optim_G.step()
        
        log_file = open('log.txt', 'w')
        log_file.write(str(epoch))
                
        # Print error and save intermediate result image and weight
        if iters % params.save_every == 0:
            print('[Epoch : %d / Iters : %d/%d] => D_loss : %f / G_GAN_loss : %f / KL_div : %f / img_recon_loss : %f / z_recon_loss : %f'\
                  %(epoch, iters, len(train_data_loader),D_loss.data[0], G_GAN_loss.data[0], KL_div.data[0], img_recon_loss.data[0], G_alone_loss.data[0]))
            
            # Save intermediate result image
            if os.path.exists(params.result_dir) is False:
                os.makedirs(params.result_dir)
                
            result_img = util.make_img(test_data_loader, G, fixed_z, img_num=params.test_img_num, img_size=params.img_size)
                
            img_name = '{epoch}_{iters}.png'.format(epoch=epoch, iters=iters)
            img_path = os.path.join(params.result_dir, img_name)
            
            torchvision.utils.save_image(result_img, img_path, nrow=params.test_img_num+1)
                
            # Save intermediate weight
            if os.path.exists(params.weight_dir) is False:
                os.makedirs(params.weight_dir)
                
            if epoch is None:
                d_cVAE_name = 'D_cVAE.pkl'
                d_cLR_name = 'D_cLR.pkl'
                g_name = 'G.pkl'
                e_name = 'E.pkl'
            else:
                d_cVAE_name = '{epochs}-{name}'.format(epochs=str(epoch), name='D_cVAE.pkl')
                d_cLR_name = '{epochs}-{name}'.format(epochs=str(epoch), name='D_cLR.pkl')
                g_name = '{epochs}-{name}'.format(epochs=str(epoch), name='G.pkl')
                e_name = '{epochs}-{name}'.format(epochs=str(epoch), name='E.pkl')
                
            torch.save(D_cVAE.state_dict(), os.path.join(params.weight_dir, d_cVAE_name))
            torch.save(D_cVAE.state_dict(), os.path.join(params.weight_dir, d_cLR_name))
            torch.save(G.state_dict(), os.path.join(params.weight_dir, g_name))
            torch.save(E.state_dict(), os.path.join(params.weight_dir, e_name))
                    
    # Save weight at the end of every epoch
    if epoch is None:
        d_cVAE_name = 'D_cVAE.pkl'
        d_cLR_name = 'D_cLR.pkl'
        g_name = 'G.pkl'
        e_name = 'E.pkl'
    else:
        d_cVAE_name = '{epochs}-{name}'.format(epochs=str(epoch), name='D_cVAE.pkl')
        d_cLR_name = '{epochs}-{name}'.format(epochs=str(epoch), name='D_cLR.pkl')
        g_name = '{epochs}-{name}'.format(epochs=str(epoch), name='G.pkl')
        e_name = '{epochs}-{name}'.format(epochs=str(epoch), name='E.pkl')
                
    torch.save(D_cVAE.state_dict(), os.path.join(params.weight_dir, d_cVAE_name))
    torch.save(D_cVAE.state_dict(), os.path.join(params.weight_dir, d_cLR_name))
    torch.save(G.state_dict(), os.path.join(params.weight_dir, g_name))
    torch.save(E.state_dict(), os.path.join(params.weight_dir, e_name))

[Epoch : 1 / Iters : 0/69283] => D_loss : 4.929877 / G_GAN_loss : 4.094715 / KL_div : 0.000412 / img_recon_loss : 8.963566 / z_recon_loss : 0.412185
[Epoch : 1 / Iters : 1000/69283] => D_loss : 2.062901 / G_GAN_loss : 1.297698 / KL_div : 0.043898 / img_recon_loss : 2.145304 / z_recon_loss : 0.492386
[Epoch : 1 / Iters : 2000/69283] => D_loss : 2.228789 / G_GAN_loss : 1.502133 / KL_div : 0.048071 / img_recon_loss : 1.377177 / z_recon_loss : 0.505225
[Epoch : 1 / Iters : 3000/69283] => D_loss : 2.058694 / G_GAN_loss : 1.129851 / KL_div : 0.066428 / img_recon_loss : 1.464293 / z_recon_loss : 0.465057
[Epoch : 1 / Iters : 4000/69283] => D_loss : 1.627337 / G_GAN_loss : 1.408136 / KL_div : 0.051107 / img_recon_loss : 1.907770 / z_recon_loss : 0.462824
[Epoch : 1 / Iters : 5000/69283] => D_loss : 2.227201 / G_GAN_loss : 1.384778 / KL_div : 0.056555 / img_recon_loss : 1.193997 / z_recon_loss : 0.290083
[Epoch : 1 / Iters : 6000/69283] => D_loss : 2.045052 / G_GAN_loss : 1.113499 / KL_div : 0.

[Epoch : 1 / Iters : 56000/69283] => D_loss : 1.979316 / G_GAN_loss : 1.314274 / KL_div : 0.176620 / img_recon_loss : 1.706052 / z_recon_loss : 0.220485
[Epoch : 1 / Iters : 57000/69283] => D_loss : 2.015038 / G_GAN_loss : 1.095156 / KL_div : 0.075100 / img_recon_loss : 0.809260 / z_recon_loss : 0.226320
[Epoch : 1 / Iters : 58000/69283] => D_loss : 1.889301 / G_GAN_loss : 1.184959 / KL_div : 0.045717 / img_recon_loss : 0.751748 / z_recon_loss : 0.341578
[Epoch : 1 / Iters : 59000/69283] => D_loss : 1.739954 / G_GAN_loss : 1.017742 / KL_div : 0.061197 / img_recon_loss : 0.786187 / z_recon_loss : 0.212689
[Epoch : 1 / Iters : 60000/69283] => D_loss : 2.509563 / G_GAN_loss : 1.484855 / KL_div : 0.071011 / img_recon_loss : 0.833016 / z_recon_loss : 0.353668
[Epoch : 1 / Iters : 61000/69283] => D_loss : 1.746247 / G_GAN_loss : 1.306478 / KL_div : 0.086113 / img_recon_loss : 2.463149 / z_recon_loss : 0.330562
[Epoch : 1 / Iters : 62000/69283] => D_loss : 1.627370 / G_GAN_loss : 0.964260 / K

[Epoch : 2 / Iters : 40000/69283] => D_loss : 1.365394 / G_GAN_loss : 1.441699 / KL_div : 0.073695 / img_recon_loss : 3.175959 / z_recon_loss : 0.251788
[Epoch : 2 / Iters : 41000/69283] => D_loss : 1.810936 / G_GAN_loss : 1.262524 / KL_div : 0.100791 / img_recon_loss : 2.128133 / z_recon_loss : 0.405757
[Epoch : 2 / Iters : 42000/69283] => D_loss : 1.813854 / G_GAN_loss : 1.143615 / KL_div : 0.087778 / img_recon_loss : 1.098434 / z_recon_loss : 0.304816
[Epoch : 2 / Iters : 43000/69283] => D_loss : 1.994809 / G_GAN_loss : 1.230770 / KL_div : 0.079784 / img_recon_loss : 0.744706 / z_recon_loss : 0.293955
[Epoch : 2 / Iters : 44000/69283] => D_loss : 1.630350 / G_GAN_loss : 1.319678 / KL_div : 0.074157 / img_recon_loss : 1.231345 / z_recon_loss : 0.388271
[Epoch : 2 / Iters : 45000/69283] => D_loss : 1.918438 / G_GAN_loss : 0.959652 / KL_div : 0.055223 / img_recon_loss : 0.907137 / z_recon_loss : 0.236476
[Epoch : 2 / Iters : 46000/69283] => D_loss : 1.763724 / G_GAN_loss : 1.238358 / K

[Epoch : 3 / Iters : 24000/69283] => D_loss : 1.530933 / G_GAN_loss : 1.539960 / KL_div : 0.076996 / img_recon_loss : 2.630671 / z_recon_loss : 0.158641
[Epoch : 3 / Iters : 25000/69283] => D_loss : 1.915392 / G_GAN_loss : 1.196874 / KL_div : 0.095022 / img_recon_loss : 1.328877 / z_recon_loss : 0.185380
[Epoch : 3 / Iters : 26000/69283] => D_loss : 1.787498 / G_GAN_loss : 1.310140 / KL_div : 0.080563 / img_recon_loss : 0.985124 / z_recon_loss : 0.252758
[Epoch : 3 / Iters : 27000/69283] => D_loss : 1.847407 / G_GAN_loss : 1.359927 / KL_div : 0.076996 / img_recon_loss : 1.534175 / z_recon_loss : 0.529938
[Epoch : 3 / Iters : 28000/69283] => D_loss : 1.385790 / G_GAN_loss : 1.590167 / KL_div : 0.058039 / img_recon_loss : 1.309481 / z_recon_loss : 0.445195
[Epoch : 3 / Iters : 29000/69283] => D_loss : 1.766001 / G_GAN_loss : 1.137619 / KL_div : 0.077486 / img_recon_loss : 0.938608 / z_recon_loss : 0.247649
[Epoch : 3 / Iters : 30000/69283] => D_loss : 1.446978 / G_GAN_loss : 1.655989 / K

[Epoch : 4 / Iters : 8000/69283] => D_loss : 1.845428 / G_GAN_loss : 0.989278 / KL_div : 0.069606 / img_recon_loss : 1.467430 / z_recon_loss : 0.219895
[Epoch : 4 / Iters : 9000/69283] => D_loss : 1.741946 / G_GAN_loss : 1.132435 / KL_div : 0.071110 / img_recon_loss : 0.852484 / z_recon_loss : 0.258622
[Epoch : 4 / Iters : 10000/69283] => D_loss : 2.363401 / G_GAN_loss : 1.305543 / KL_div : 0.083337 / img_recon_loss : 0.548159 / z_recon_loss : 0.433360
[Epoch : 4 / Iters : 11000/69283] => D_loss : 1.551252 / G_GAN_loss : 1.471780 / KL_div : 0.073371 / img_recon_loss : 1.988855 / z_recon_loss : 0.251331
[Epoch : 4 / Iters : 12000/69283] => D_loss : 1.895714 / G_GAN_loss : 1.726649 / KL_div : 0.079905 / img_recon_loss : 1.719093 / z_recon_loss : 0.313180
[Epoch : 4 / Iters : 13000/69283] => D_loss : 1.900489 / G_GAN_loss : 1.434177 / KL_div : 0.087800 / img_recon_loss : 1.753329 / z_recon_loss : 0.221930
[Epoch : 4 / Iters : 14000/69283] => D_loss : 1.601900 / G_GAN_loss : 1.602072 / KL_

[Epoch : 4 / Iters : 62000/69283] => D_loss : 1.939408 / G_GAN_loss : 1.186298 / KL_div : 0.078112 / img_recon_loss : 0.998048 / z_recon_loss : 0.257412
[Epoch : 4 / Iters : 63000/69283] => D_loss : 1.855092 / G_GAN_loss : 1.101893 / KL_div : 0.073188 / img_recon_loss : 1.276701 / z_recon_loss : 0.272768
[Epoch : 4 / Iters : 64000/69283] => D_loss : 1.730575 / G_GAN_loss : 1.374205 / KL_div : 0.065565 / img_recon_loss : 0.471770 / z_recon_loss : 0.389275
[Epoch : 4 / Iters : 65000/69283] => D_loss : 1.797932 / G_GAN_loss : 0.904813 / KL_div : 0.077902 / img_recon_loss : 0.871231 / z_recon_loss : 0.416313
[Epoch : 4 / Iters : 66000/69283] => D_loss : 2.166176 / G_GAN_loss : 1.171481 / KL_div : 0.074033 / img_recon_loss : 0.463492 / z_recon_loss : 0.202976
[Epoch : 4 / Iters : 67000/69283] => D_loss : 2.574461 / G_GAN_loss : 0.990283 / KL_div : 0.135515 / img_recon_loss : 0.609695 / z_recon_loss : 0.346596
[Epoch : 4 / Iters : 68000/69283] => D_loss : 2.120769 / G_GAN_loss : 1.548858 / K

[Epoch : 5 / Iters : 46000/69283] => D_loss : 1.415403 / G_GAN_loss : 1.209459 / KL_div : 0.097688 / img_recon_loss : 2.134909 / z_recon_loss : 0.338627
[Epoch : 5 / Iters : 47000/69283] => D_loss : 2.288019 / G_GAN_loss : 1.395247 / KL_div : 0.079189 / img_recon_loss : 0.718144 / z_recon_loss : 0.183946
[Epoch : 5 / Iters : 48000/69283] => D_loss : 1.666379 / G_GAN_loss : 1.676416 / KL_div : 0.098462 / img_recon_loss : 1.143301 / z_recon_loss : 0.191846
[Epoch : 5 / Iters : 49000/69283] => D_loss : 1.715250 / G_GAN_loss : 1.447090 / KL_div : 0.099584 / img_recon_loss : 1.595857 / z_recon_loss : 0.279610
[Epoch : 5 / Iters : 50000/69283] => D_loss : 1.830874 / G_GAN_loss : 1.380952 / KL_div : 0.088937 / img_recon_loss : 0.920234 / z_recon_loss : 0.235542
[Epoch : 5 / Iters : 51000/69283] => D_loss : 1.768467 / G_GAN_loss : 1.330717 / KL_div : 0.095159 / img_recon_loss : 1.451729 / z_recon_loss : 0.417214
[Epoch : 5 / Iters : 52000/69283] => D_loss : 1.448531 / G_GAN_loss : 1.641308 / K

[Epoch : 6 / Iters : 31000/69283] => D_loss : 1.611332 / G_GAN_loss : 1.278110 / KL_div : 0.086807 / img_recon_loss : 0.917786 / z_recon_loss : 0.149673
[Epoch : 6 / Iters : 32000/69283] => D_loss : 1.579077 / G_GAN_loss : 1.705955 / KL_div : 0.131320 / img_recon_loss : 1.172971 / z_recon_loss : 0.192251
[Epoch : 6 / Iters : 33000/69283] => D_loss : 1.903393 / G_GAN_loss : 1.125727 / KL_div : 0.134622 / img_recon_loss : 1.210595 / z_recon_loss : 0.167952
[Epoch : 6 / Iters : 34000/69283] => D_loss : 1.908695 / G_GAN_loss : 0.758290 / KL_div : 0.096982 / img_recon_loss : 1.185851 / z_recon_loss : 0.272793
[Epoch : 6 / Iters : 35000/69283] => D_loss : 1.765776 / G_GAN_loss : 1.391788 / KL_div : 0.130500 / img_recon_loss : 2.462798 / z_recon_loss : 0.305723
[Epoch : 6 / Iters : 36000/69283] => D_loss : 1.930157 / G_GAN_loss : 0.996830 / KL_div : 0.120735 / img_recon_loss : 2.325854 / z_recon_loss : 0.344680
[Epoch : 6 / Iters : 37000/69283] => D_loss : 2.053226 / G_GAN_loss : 1.137884 / K

[Epoch : 7 / Iters : 15000/69283] => D_loss : 1.672656 / G_GAN_loss : 1.357937 / KL_div : 0.099190 / img_recon_loss : 0.851951 / z_recon_loss : 0.421098
[Epoch : 7 / Iters : 16000/69283] => D_loss : 1.528891 / G_GAN_loss : 1.255315 / KL_div : 0.108115 / img_recon_loss : 1.032838 / z_recon_loss : 0.373251
[Epoch : 7 / Iters : 17000/69283] => D_loss : 1.598080 / G_GAN_loss : 1.197066 / KL_div : 0.133242 / img_recon_loss : 1.230355 / z_recon_loss : 0.368590
[Epoch : 7 / Iters : 18000/69283] => D_loss : 1.849680 / G_GAN_loss : 1.329989 / KL_div : 0.099700 / img_recon_loss : 0.960997 / z_recon_loss : 0.253578
[Epoch : 7 / Iters : 19000/69283] => D_loss : 1.634687 / G_GAN_loss : 1.253790 / KL_div : 0.099808 / img_recon_loss : 1.634400 / z_recon_loss : 0.161029
[Epoch : 7 / Iters : 20000/69283] => D_loss : 1.972495 / G_GAN_loss : 1.667873 / KL_div : 0.103717 / img_recon_loss : 1.884482 / z_recon_loss : 0.343196
[Epoch : 7 / Iters : 21000/69283] => D_loss : 1.758100 / G_GAN_loss : 2.309681 / K

[Epoch : 7 / Iters : 69000/69283] => D_loss : 1.994373 / G_GAN_loss : 0.923183 / KL_div : 0.112366 / img_recon_loss : 1.119123 / z_recon_loss : 0.232857
[Epoch : 8 / Iters : 0/69283] => D_loss : 1.719651 / G_GAN_loss : 1.317060 / KL_div : 0.118583 / img_recon_loss : 1.782731 / z_recon_loss : 0.226584
[Epoch : 8 / Iters : 1000/69283] => D_loss : 1.478943 / G_GAN_loss : 1.422956 / KL_div : 0.142260 / img_recon_loss : 1.090229 / z_recon_loss : 0.105786
[Epoch : 8 / Iters : 2000/69283] => D_loss : 1.767802 / G_GAN_loss : 1.201748 / KL_div : 0.147973 / img_recon_loss : 1.032090 / z_recon_loss : 0.133323
[Epoch : 8 / Iters : 3000/69283] => D_loss : 2.205485 / G_GAN_loss : 1.124833 / KL_div : 0.092924 / img_recon_loss : 0.507901 / z_recon_loss : 0.386854
[Epoch : 8 / Iters : 4000/69283] => D_loss : 1.840598 / G_GAN_loss : 1.392381 / KL_div : 0.084596 / img_recon_loss : 0.617270 / z_recon_loss : 0.216285
[Epoch : 8 / Iters : 5000/69283] => D_loss : 1.626315 / G_GAN_loss : 1.244693 / KL_div : 0